<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#EDA" data-toc-modified-id="EDA-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>EDA</a></span><ul class="toc-item"><li><span><a href="#Задача-1" data-toc-modified-id="Задача-1-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Задача 1</a></span></li><li><span><a href="#Задача-2" data-toc-modified-id="Задача-2-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Задача 2</a></span></li><li><span><a href="#Задача-3" data-toc-modified-id="Задача-3-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Задача 3</a></span></li><li><span><a href="#Задача-4" data-toc-modified-id="Задача-4-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Задача 4</a></span></li><li><span><a href="#Задача-5" data-toc-modified-id="Задача-5-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Задача 5</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# SQL

Компания купила крупный сервис для чтения книг по подписке.

**Цель исследования** - проанализировать базу данных для формирования ценностного предложения для нового продукта.

**Описание данных:**
1. Таблица книг `books`:
    * `book_id` — идентификатор книги (первичный ключ)
    * `author_id` — идентификатор автора (внешний ключ)
    * `title` — название книги
    * `num_pages` — количество страниц
    * `publication_date` — дата публикации книги
    * `publisher_id` — идентификатор издателя (внешний ключ)


2. Таблица авторов `authors`:
    * `author_id` — идентификатор автора (первичный ключ)
    * `author` — имя автора


3. Таблица издательств `publishers`:
    * `publisher_id` — идентификатор издательства (первичный ключ)
    * `publisher` — название издательства


4. Таблица пользовательских оценок книг `ratings`:
    * `rating_id` — идентификатор оценки (первичный ключ)
    * `book_id` — идентификатор книги (внешний ключ)
    * `username` — имя пользователя, оставившего оценку
    * `rating` — оценка книги


5. Таблица пользовательских обзоров на книги `reviews`**
    * `review_id` — идентификатор обзора (первичный ключ)
    * `book_id` — идентификатор книги (внешний ключ)
    * `username` — имя пользователя, написавшего обзор
    * `text` — текст обзора
    
**Ход исследования:**
1. Обзор данных
2. EDA
3. Выводы

## Обзор данных

Для начала загрузим необходимые библиотеки:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Подключимся к базе данных:

In [2]:
# устанавливаем параметры:
db_config = {'user': '//censored//', # имя пользователя
            'pwd': '//censored//', # пароль
            'host': '//censored//',
            'port': 6432, # порт подключения
            'db': '//censored//'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                        db_config['pwd'],
                                                        db_config['host'],
                                                        db_config['port'],
                                                        db_config['db'])

# сохраняем коннектор:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Теперь перейдём к исследованию таблиц: выведем первые строки каждой из них.

In [3]:
query = '''
            SELECT * FROM books
            LIMIT 10
        '''

display(pd.io.sql.read_sql(query, con = engine))

query = '''
            SELECT COUNT(book_id) AS total_number_of_books
            FROM books
        '''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


,total_number_of_books
0,1000


Так, первичным ключом таблицы `books` является идентификатор книги - `book_id`. В таблице также есть два внешних ключа - `author_id` для связи с таблицей `authors` и `publisher_id` для связи с таблицей `publishers`. Всего в таблице о книгах содержится записи о 1000 книг.

Проверим, книги каких годов выпуска представлены в каталоге сервиса:

In [4]:
query = '''
            SELECT CAST(MIN(EXTRACT(YEAR FROM CAST(publication_date AS timestamp))) AS int),
                   CAST(MAX(EXTRACT(YEAR FROM CAST(publication_date AS timestamp))) AS int)
            FROM books
        '''

pd.io.sql.read_sql(query, con = engine)

,min,max
0,1952,2020


Так, в сервисе представлены книги 1952 - 2020 гг. выпуска.

Посмотрим информацию о таблице `authors`:

In [5]:
query = '''
            SELECT * FROM authors
            LIMIT 10
        '''

display(pd.io.sql.read_sql(query, con = engine))

query = '''
            SELECT COUNT(author_id) AS total_number_of_authors
            FROM authors
        '''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


,total_number_of_authors
0,636


В таблице с авторами содержится информация о 636 авторах. В таблице всего два столбца - идентификатор автора (первичный ключ) и его имя.

Перейдём к таблице издательств:

In [6]:
query = '''
            SELECT * FROM publishers
            LIMIT 10
        '''

display(pd.io.sql.read_sql(query, con = engine))

query = '''
            SELECT COUNT(publisher_id) AS total_number_of_publishers
            FROM publishers
        '''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


,total_number_of_publishers
0,340


В таблице `publishers` присутствует информация о 340 издательствах. Её структура похожа на структуру таблицы `authors`.

Рассмотрим таблицы `ratings` и `reviews`:

In [7]:
query = '''
            SELECT * FROM ratings
            LIMIT 10
        '''

display(pd.io.sql.read_sql(query, con = engine))

query = '''
            SELECT COUNT(rating_id) AS total_number_of_ratings
            FROM ratings
        '''

display(pd.io.sql.read_sql(query, con = engine))

query = '''
            SELECT * FROM reviews
            LIMIT 10
        '''

display(pd.io.sql.read_sql(query, con = engine))

query = '''
            SELECT COUNT(review_id) AS total_number_of_reviews
            FROM reviews
        '''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


,total_number_of_ratings
0,6456


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


,total_number_of_reviews
0,2793


В обеих таблицах есть внешний ключ `book_id`, отсылающий к таблице `books`. Всего пользователи оценили книги 6456 раз и написали 2793 обзора.

Посмотрим, в каких пределах варьируются возможные значения рейтинга книги:

In [8]:
query = '''
            SELECT MIN(rating),
                   MAX(rating)
            FROM ratings
        '''

pd.io.sql.read_sql(query, con = engine)

,min,max
0,1,5


Так, оценка пользователей может варьироваться в пределах от 1 до 5.

Можно сделать вывод, что в сервисе представлена 1000 книг 1952-2020 гг. выпуска, написанных 636 разными авторами, опубликованная 340 издательствами. Пользователи сервиса поставили 6456 оценок от 1 до 5, написали 2793 обзора.

Теперь, когда общее представление о каталоге сервиса получено, можно приступать к исследовательского анализу данных.

## EDA

### Задача 1

Посчитаем, сколько книг вышло в XXI веке, т. е. после 1 января 2000 г. (включительно):

In [9]:
query = '''
            SELECT COUNT(book_id) AS this_century_books
            FROM books
            WHERE CAST(publication_date AS timestamp) >= '2000-01-01'
        '''

pd.io.sql.read_sql(query, con = engine)

,this_century_books
0,821


Так, из 1000 книг в каталоге 821 вышла в свет в XXI веке, т. е. в сервисе представлены в основном современная литература или переизданные произведения.

### Задача 2

Посчитаем для каждой книги количество обзоров и среднюю оценку:

In [10]:
query = '''
            WITH re AS
            (SELECT book_id,
                    COUNT(review_id) AS reviews_number
            FROM reviews
            GROUP BY book_id),
            
            ra AS 
            (SELECT book_id,
                    AVG(rating) AS avg_rating
            FROM ratings
            GROUP BY book_id)
            
            -- выведем Топ-10 книг с наибольшим количеством обзоров: --
            
            SELECT b.book_id,
                   b.title,
                   re.reviews_number,
                   ra.avg_rating
            FROM books AS b
            JOIN re ON re.book_id = b.book_id
            JOIN ra ON ra.book_id = b.book_id
            ORDER BY re.reviews_number DESC, ra.avg_rating DESC
            LIMIT 10
        '''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,reviews_number,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,963,Water for Elephants,6,3.977273


Так, у книг, на которые пользователи написали наибольшее количество рецензий, далеко не самые высокие средние рейтинги. Например, Топ-10 возглавляет первая книга из серии "Сумерки" с 7 рецензиями, средняя оценка которой равна 3.7. Однако все книги из рейтинга очень популярные: здесь присутствует много книг о Гарри Поттере, книги Толкиена и др. В целом можно сказать, что у наиболее популярных книг средний рейтинг не будет максимальным (поскольку среди большого количества читателей вероятность найти того, кому книга не понравилась, намного выше).

### Задача 3

Теперь посмотрим, какое издательство является самым крупным. Для этого найдём то, которое выпустило наибольшее число книг толще 50 страниц (исключаем из анализа брошюры):

In [11]:
query = '''
            SELECT p.publisher,
                   COUNT(b.book_id) AS total_books
            FROM books AS b
            JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher_id
            ORDER BY total_books DESC
            LIMIT 1
        '''

pd.io.sql.read_sql(query, con = engine)

,publisher,total_books
0,Penguin Books,42


Наибольшее число книг (42) выпустило издательство Penguin Books, тем не менее это всё равно очень мало. Можно выделить две причины: 1) сервис сотрудничает только с маленькими издательствами (однако это маловероятно, поскольку в его коллекции присутствуют известные бестселлеры), 2) сервис закупает небольшое количество книг у разных издательств (это кажется наиболее вероятным). Такой подход говорит о том, что коллекция сервиса, хотя и небольшая, но разнообразная.

### Задача 4

Определим автора с самой высокой средней оценкой книг (чтобы можно выделить более объективное мнение, будем учитывать только книги с 50 и более оценками):

In [12]:
query = '''
            SELECT a.author,
                   AVG(r.rating) AS avg_rating
            FROM authors AS a
            JOIN books AS b ON a.author_id = b.author_id
            JOIN ratings AS r ON b.book_id = r.book_id
            WHERE r.book_id IN (SELECT book_id
                                FROM ratings
                                GROUP BY book_id
                                HAVING COUNT(rating_id) >= 50)
            GROUP BY a.author_id
            ORDER BY avg_rating DESC
            LIMIT 1
        '''

pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Так, наиболее высокая средняя оценка в сервисе (4.3) принадлежит дуэту Джоан Роулинг и Мэри Грандпре, создавшим книги о Гарри Поттере.

### Задача 5

Проанализируем поведение активных пользователей: посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок:

In [15]:
query = '''
            SELECT AVG(reviews_number) AS avg_reviews FROM (
                                             SELECT DISTINCT username,
                                                    COUNT(review_id) AS reviews_number
                                             FROM reviews
                                             WHERE username IN (SELECT username
                                                                FROM ratings
                                                                GROUP BY username
                                                                HAVING COUNT(rating_id) >= 50) 
                                             GROUP BY username) re
        '''

pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24.222222


Так, в среднем такие пользователи пишут около 24 рецензий.

## Выводы

В сервисе представлена **1000 книг 1952-2020 гг. выпуска**, написанных **636 разными авторами**, опубликованная **340 издательствами**. Пользователи сервиса поставили **6456 оценок** от 1 до 5, написали **2793 обзора**.

Кроме того, в ходе EDA было выяснено, что:
* Более **80% книг**, представленных в сервисе, было **издано в XXI веке**.
* Книги, на которые пользователи написали **наибольшее количество рецензий**, являются **мировыми бестселлерами**, но имеют **не максимальные средние рейтинги**.
* В сервисе представлена **разнообразная литература**.
* Автор с самым высоким средним рейтингом в сервисе - **Джоан Роулинг**.
* Пользователи, которые ставят **много оценок** (больше 50), в среднем пишут **около 24 рецензий**.